<a href="https://colab.research.google.com/github/thomasavare/DNLP-project/blob/master/mesh_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install awscli
!aws s3 sync --no-sign-request s3://ai2-s2-research-public/specter/scidocs/ data/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

train_path, test_path = '/content/data/mesh/train.csv', '/content/data/mesh/test.csv'
train_df, test_df = pd.read_csv(train_path), pd.read_csv(test_path)
train_df

,pid,class_label
0,dda7899c46b7764ed16ab3092f4f9476a9cecedf,9
1,16d289c10d9834d6fcb2870ba0d6cb5fd839e9a3,7
2,a1337bbf7cff59dcd9d63600ac80eb90f191a26e,8
3,3b24a936f546fd65a9bfbeb79496a5edc6d0d419,9
4,be0a5a24498086d9544d0c9dd328a9a76a8aadb2,8
...,...,...
16473,f85e9cbd0ebbdc89873b945eb98d22f7d9211957,9
16474,c4165de55761f1bb97c8e7c5224c7617a061d1bc,8
16475,993ec23465dce0d7067d674b5b21426d716f372b,9
16476,1437f6adbedcfc60432df230128383d98997f93e,2


In [ ]:
embeddings_df = pd.read_json('/content/drive/MyDrive/DNLP-project/embeddings_metadata_mag_mesh.json', orient='index')

In [ ]:
train_merged_df, test_merged_df = pd.merge(embeddings_df, train_df, how='inner', left_on='paper_id', right_on='pid'), pd.merge(embeddings_df, test_df, how='inner', left_on='paper_id', right_on='pid')
train_merged_df, test_merged_df = train_merged_df[['paper_id', 'embedding', 'class_label']], test_merged_df[['paper_id', 'embedding', 'class_label']]

In [ ]:
import numpy as np

x_train, y_train = [], []
x_test, y_test = [], []

i = 0
for _, embedding, class_label in train_merged_df.values:
  x_train.append(embedding)
  y_train.append(class_label)

x_train = np.array(x_train)
y_train = np.array(y_train)

for _, embedding, class_label in test_merged_df.values:
  x_test.append(embedding)
  y_test.append(class_label)

x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import pickle

Linear SVC

In [ ]:
from sklearn.svm import LinearSVC

parameters = {'C': np.logspace(-4, 2, 7)}
svc = LinearSVC(max_iter=50000)
clf1 = GridSearchCV(svc, parameters, cv=3)

In [ ]:
clf1.fit(x_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warning

GridSearchCV(cv=3, estimator=LinearSVC(max_iter=50000),
             param_grid={'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])})

In [ ]:
y_pred = clf1.predict(x_test)
clf1.score(x_test, y_test), np.mean(y_test == y_pred)

(0.8444948921679909, 0.8444948921679909)

In [ ]:
filename = '/content/drive/MyDrive/DNLP-project/linearSVC_model_mesh.sav'
pickle.dump(clf1, open(filename, 'wb'))

SVC

In [ ]:
from sklearn.svm import SVC

parameters = {'C': np.logspace(-4, 2, 7)}
svc = SVC()
clf2 = GridSearchCV(svc, parameters, cv=3)

```
from sklearn.svm import SVC

parameters = {'C': np.logspace(-4, 2, 7)}
svc = SVC()
clf2 = GridSearchCV(svc, parameters, cv=3, scoring=make_scorer(f1_score, average="macro"))
```
used tpo have f1 scoring function

In [ ]:
clf2.fit(x_train, y_train)

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])})

In [ ]:
y_pred = clf2.predict(x_test)
clf2.score(x_test, y_test), np.mean(y_test == y_pred)

(0.8476163450624291, 0.8476163450624291)

In [ ]:
filename = '/content/drive/MyDrive/DNLP-project/SVC_model_mesh.sav'
pickle.dump(clf2, open(filename, 'wb'))

Gaussian naive bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

In [ ]:
gnb.fit(x_train, y_train)

GaussianNB()

In [ ]:
y_pred = gnb.predict(x_test)
gnb.score(x_test, y_test), f1_score(y_test, y_pred, average='macro')

(0.7122587968217934, 0.681885251722862)

In [ ]:
filename = '/content/drive/MyDrive/DNLP-project/GNB_model_mesh.sav'
pickle.dump(gnb, open(filename, 'wb'))

Knn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knb = KNeighborsClassifier()

In [ ]:
knb.fit(x_train, y_train)

KNeighborsClassifier()

In [ ]:
y_pred = knb.predict(x_test)
knb.score(x_test, y_test), f1_score(y_test, y_pred, average="macro")

(0.8073212258796821, 0.7777932401154167)

In [ ]:
filename = '/content/drive/MyDrive/DNLP-project/Knn_model_mesh.sav'
pickle.dump(knb, open(filename, 'wb'))

SGD

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()

In [ ]:
sgd.fit(x_train, y_train)

SGDClassifier()

In [ ]:
y_pred = sgd.predict(x_test)
sgd.score(x_test, y_test), f1_score(y_test, y_pred, average="macro")

(0.815550510783201, 0.790758429159007)

In [ ]:
filename = '/content/drive/MyDrive/DNLP-project/SGD_model_mesh.sav'
pickle.dump(knb, open(filename, 'wb'))